In [ ]:
import covidcast
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

%cd ../code
from model import Model

%cd ../notebooks

In [ ]:
#Load data

start_date = datetime(2020, 10, 15)
end_date = datetime(2020, 12, 15)
    
X_df = covidcast.signal(
        'fb-survey', 
        'smoothed_cli',
        start_date, end_date,
        geo_type='state')

Y_df = covidcast.signal(
        'indicator-combination',
        'confirmed_7dav_incidence_num',
        start_date, end_date,
        geo_type='state')

In [ ]:
#Process data for use in Tensorflow

states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga',
       'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me',
       'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm',
       'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn',
       'tx', 'ut', 'va', 'vt', 'wa', 'wi', 'wv', 'wy']

df = X_df.sort_values(['geo_value', 'time_value'])
df = df[df['geo_value'].isin(states)]
X = pd.pivot_table(df,index='geo_value',columns=df.groupby(['geo_value']).cumcount().add(1), values='value').values
X = tf.expand_dims(X,axis=0)
X = tf.expand_dims(X,axis=-1)
X = tf.cast(X, dtype=tf.float32)

df = Y_df.sort_values(['geo_value', 'time_value'])
df = df[df['geo_value'].isin(states)]
Y = pd.pivot_table(df,index='geo_value',columns=df.groupby(['geo_value']).cumcount().add(1), values='value').values
Y = tf.expand_dims(Y,axis=0)
Y = tf.expand_dims(Y,axis=-1)
Y = tf.cast(Y, dtype=tf.float32)

In [ ]:
#Initialize model and parameters

#You can apply constraints directly to layers by setting
#the kernel_constrain parameter for a specific layer
#
#https://www.tensorflow.org/api_docs/python/tf/keras/constraints

p=30
kernel_constraint = tf.keras.constraints.NonNeg()
lr=1

model = Model(p=p, kernel_constraint=kernel_constraint)
optimizer = tf.keras.optimizers.Adam(lr=lr)

def MSE(y_true, y_pred):
    return tf.reduce_mean((y_true-y_pred)**2)

loss_object=MSE

In [ ]:
#Training
@tf.function
def train_step(X,Y):
    assert X.shape[1] == Y.shape[1], "Size of X and Y should be the same shape but found, {} vs {}".format(
        X.shape[1], Y.shape[1])

    X_padded = tf.pad(
        X,
        paddings =[[0, 0], [0, 0], [p-1, 0], [0, 0]],
    )

    with tf.GradientTape() as tape:
        Y_hat = model(X_padded, training=True)
        loss = loss_object(Y, Y_hat)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))


EPOCHS=1000
MODEL_PATH="<PATH>" #Change path 
best_loss = tf.constant(np.inf) 

for epoch in range(EPOCHS):
    train_step(X,Y)

    X_padded = tf.pad(
        X,
        paddings =[[0, 0], [0, 0], [p-1, 0], [0, 0]],
    )
    Y_hat = model(X_padded, training=True)
    loss = loss_object(Y, Y_hat)

    if loss<best_loss:
        best_loss = loss
        model.save_weights(MODEL_PATH)

    print("Epoch: {}, Current Loss: {}, Best Loss: {}".format(epoch, loss, best_loss))

In [ ]:
#Evaluation
model.load_weights(MODEL_PATH).expect_partial() #load best weights from training